In [2]:
import PyPDF2
import re
import gradio as gr

# Predefined IT job roles and skills
JOB_ROLES = {
    "Data Scientist": ["python", "machine learning", "statistics", "sql", "data analysis", "r", "pandas", "numpy", "data visualization", "scikit-learn", "probability", "mathematics"],
    "Software Engineer": ["python", "java", "c++", "software development", "git", "javascript", "agile", "debugging", "oop", "api development", "unit testing", "design patterns"],
    "Project Manager": ["project management", "leadership", "communication", "budgeting", "scheduling", "agile", "scrum", "risk management", "stakeholder management", "ms project", "team coordination"],
    "Machine Learning Engineer": ["python", "machine learning", "deep learning", "tensorflow", "pytorch", "numpy", "scikit-learn", "data preprocessing", "model deployment", "computer vision", "optimization"],
    "AI Researcher": ["python", "ai", "nlp", "deep learning", "research", "tensorflow", "pytorch", "mathematics", "algorithms", "data modeling", "publication", "statistics"],
    "Network Engineer": ["networking", "cisco", "tcp/ip", "firewalls", "routing", "switching", "linux", "vpn", "network security", "troubleshooting", "wireshark"],
    "Cybersecurity Analyst": ["cybersecurity", "networking", "linux", "penetration testing", "firewalls", "ethical hacking", "incident response", "security audits", "malware analysis", "siem"],
    "Web Developer": ["html", "css", "javascript", "php", "react", "node.js", "web design", "rest apis", "bootstrap", "sql", "responsive design"],
    "Database Administrator": ["sql", "mysql", "postgresql", "database management", "backup", "recovery", "performance tuning", "data modeling", "oracle", "security", "indexing"],
    "Systems Administrator": ["linux", "windows", "networking", "server management", "cloud", "scripting", "active directory", "virtualization", "troubleshooting", "backup", "monitoring"],
    "IT Support Specialist": ["troubleshooting", "windows", "networking", "customer service", "hardware", "software installation", "help desk", "ticketing systems", "remote support", "basic scripting"],
    "DevOps Engineer": ["docker", "kubernetes", "ci/cd", "aws", "linux", "git", "scripting", "jenkins", "ansible", "monitoring", "infrastructure as code"],
    "Cloud Architect": ["aws", "azure", "google cloud", "cloud computing", "architecture", "security", "migration", "scalability", "networking", "cost management", "devops"],
    "Business Analyst": ["data analysis", "communication", "sql", "requirements gathering", "process modeling", "stakeholder management", "excel", "business intelligence", "uml", "documentation"],
    "Quality Assurance Tester": ["testing", "selenium", "automation", "bug tracking", "test cases", "manual testing", "jira", "regression testing", "test planning", "api testing"],
    "Information Security Engineer": ["cybersecurity", "encryption", "firewalls", "incident response", "network security", "vulnerability assessment", "siem", "identity management", "compliance", "penetration testing"],
    "UX Designer": ["ui/ux", "prototyping", "user research", "wireframing", "design tools", "adobe xd", "figma", "usability testing", "interaction design", "visual design"],
    "IT Manager": ["leadership", "project management", "budgeting", "team management", "it strategy", "vendor management", "risk management", "communication", "policy development", "resource allocation"],
    "Data Engineer": ["python", "sql", "etl", "big data", "hadoop", "spark", "data pipelines", "data warehousing", "cloud", "apache kafka", "data modeling"],
    "Game Developer": ["c++", "unity", "unreal engine", "game design", "3d modeling", "c#", "physics", "animation", "gameplay programming", "multiplayer development"],
    "Mobile Developer": ["swift", "kotlin", "java", "mobile development", "ios", "android", "react native", "ui design", "api integration", "debugging"],
    "Embedded Systems Engineer": ["c", "c++", "embedded systems", "microcontrollers", "linux", "hardware", "firmware", "real-time systems", "debugging", "circuit design"],
    "Blockchain Developer": ["blockchain", "solidity", "ethereum", "smart contracts", "cryptography", "python", "web3.js", "distributed systems", "security", "consensus algorithms"],
    "Site Reliability Engineer": ["linux", "python", "monitoring", "aws", "reliability", "incident response", "automation", "kubernetes", "performance tuning", "scripting"],
    "Full Stack Developer": ["javascript", "python", "html", "css", "node.js", "react", "sql", "rest apis", "git", "cloud", "agile"],
    "Technical Writer": ["technical writing", "documentation", "communication", "markdown", "api documentation", "user manuals", "editing", "research", "content management", "version control"],
    "Robotics Engineer": ["python", "c++", "robotics", "ros", "control systems", "sensors", "computer vision", "embedded systems", "mechanical design", "automation"],
    "AR/VR Developer": ["unity", "c#", "unreal engine", "ar/vr", "3d modeling", "augmented reality", "virtual reality", "javascript", "spatial computing", "game design"],
    "IoT Developer": ["python", "c", "iot", "networking", "sensors", "cloud", "mqtt", "embedded systems", "data processing", "security"]
}

In [6]:
def extract_text_from_pdf(pdf_file):
    with open(pdf_file, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
        text = re.sub(r'[^\w\s@.-]', ' ', text)
        text = re.sub(r'\s+', ' ', text).strip()
        return text
pdf_path = "G Kamesh Resume.pdf"  # Replace with your file path
resume_text = extract_text_from_pdf(pdf_path)
print(resume_text[:500])

G Kamesh ap- arker-altHyderabad envel pekamesh743243@gmail.com phone-alt6300743243 githubkamesh1410 gl be My Portfolio external-link-alt Summary Applied Scientist Expertise in AI NLP and Machine Learning for Business Solutions Results-driven Machine Learning professional with expertise in AI NLP and LLMs specializing in developing and de- ploying scalable ML models for real-world applications. Proficient in Python data analysis with hands-on experience in building deep learning models optimizing


In [11]:
def resume_chatbot(pdf_file, job_role):
    resume_text = extract_text_from_pdf(pdf_file)
    text_lower = resume_text.lower()

    job_role_lower = job_role.lower()
    job_roles_lower = {key.lower(): key for key in JOB_ROLES.keys()}
    if job_role_lower not in job_roles_lower:
        return f"Error: Job role '{job_role}' not recognized. Available roles: {', '.join(JOB_ROLES.keys())}"
    original_job_role = job_roles_lower[job_role_lower]
    job_skills = JOB_ROLES[original_job_role]

    mistakes = []
    if len(resume_text) < 100:
        mistakes.append("Resume is too short or text extraction failed.")

    # Stricter section detection
    sections = {
        "contact": r"\b(name|email|phone|github|portfolio)\b",
        "summary": r"\b(summary|objective|profile)\b",
        "skills": r"\b(skills|expertise|abilities|technologies)\b",
        "experience": r"\b(experience|work|job|employment|professional)\b",
        "education": r"\b(education|degree|university|college|school)\b"
    }
    section_positions = {}
    for section, pattern in sections.items():
        match = re.search(pattern, text_lower)
        if match:
            section_positions[section] = match.start()

    structure_feedback = []
    expected_order = ["contact", "summary", "skills", "experience", "education"]
    detected_sections = list(section_positions.keys())
    missing_sections = [s for s in expected_order if s not in detected_sections]
    if missing_sections:
        structure_feedback.append(f"Missing sections: {', '.join(missing_sections)}")
    if detected_sections:
        sorted_sections = sorted(detected_sections, key=lambda x: section_positions[x])
        ideal_order = [s for s in expected_order if s in detected_sections]
        if sorted_sections != ideal_order:
            structure_feedback.append(f"Sections out of order. Detected: {', '.join(sorted_sections)}. Recommended: {', '.join(ideal_order)}")

    detected_skills = []
    for skill in job_skills:
        if re.search(r'\b' + re.escape(skill) + r'\b', text_lower):
            detected_skills.append(skill)
    matched_skills = len(detected_skills)
    total_skills = len(job_skills)
    score = (matched_skills / total_skills) * 100 if total_skills > 0 else 0
    if len(re.findall(r"\b(header|footer|image|table)\b", text_lower)) > 0:
        score -= 20
    score = max(0, min(100, score))
    missing_skills = [skill for skill in job_skills if skill not in detected_skills]

    skill_feedback = []
    critical_skills = job_skills[:5]
    for skill in detected_skills:
        relevance = "Critical" if skill in critical_skills else "Supplementary"
        skill_feedback.append(f"- {skill}: {relevance} skill detected")
    if not detected_skills:
        skill_feedback.append("- No relevant skills detected")

    score_bar = "[" + "█" * int(score // 10) + " " * (10 - int(score // 10)) + "]"
    output = "Resume Analysis Report:\n"
    output += f"Job Role: {original_job_role}\n"
    output += f"ATS Score: {score:.1f}% {score_bar}\n"

    output += "\nSummary:\n"
    output += "Resume Structure Analysis:\n"
    if structure_feedback:
        output += "Issues Detected:\n" + "\n".join([f"- {f}" for f in structure_feedback]) + "\n"
        output += "Recommended Order: Contact → Summary → Skills → Experience → Education\n"
    else:
        output += "Issues Detected: None (Structure follows recommended order)\n"
    if mistakes:
        output += "General Issues:\n" + "\n".join([f"- {m}" for m in mistakes]) + "\n"
    output += "Skill Relevance Summary:\n"
    output += "\n".join(skill_feedback) + "\n"
    if missing_skills:
        output += f"Suggested Skills to Add (out of {total_skills} expected):\n" + "\n".join([f"- {s}" for s in missing_skills])
    else:
        output += "Suggested Skills to Add: None"
    
    return output

# Test with your resume text
print(resume_chatbot(pdf_path, "data scientist"))

Resume Analysis Report:
Job Role: Data Scientist
ATS Score: 75.0% [███████   ]

Summary:
Resume Structure Analysis:
Issues Detected: None (Structure follows recommended order)
Skill Relevance Summary:
- python: Critical skill detected
- machine learning: Critical skill detected
- sql: Critical skill detected
- data analysis: Critical skill detected
- pandas: Supplementary skill detected
- numpy: Supplementary skill detected
- data visualization: Supplementary skill detected
- scikit-learn: Supplementary skill detected
- mathematics: Supplementary skill detected
Suggested Skills to Add (out of 12 expected):
- statistics
- r
- probability


In [17]:
def gradio_resume_chatbot(pdf_file, job_role):
    resume_text = extract_text_from_pdf(pdf_file.name)
    text_lower = resume_text.lower()

    job_role_lower = job_role.lower()
    job_roles_lower = {key.lower(): key for key in JOB_ROLES.keys()}
    if job_role_lower not in job_roles_lower:
        return f"<div style='font-family: Arial, sans-serif; padding: 20px;'><p style='color: red;'>Error: Job role '{job_role}' not recognized. Available roles: {', '.join(JOB_ROLES.keys())}</p></div>"
    original_job_role = job_roles_lower[job_role_lower]
    job_skills = JOB_ROLES[original_job_role]

    mistakes = []
    if len(resume_text) < 100:
        mistakes.append("Resume is too short or text extraction failed.")

    sections = {
        "contact": r"\b(name|email|phone|github|portfolio)\b",
        "summary": r"\b(summary|objective|profile)\b",
        "skills": r"\b(skills|expertise|abilities|technologies)\b",
        "experience": r"\b(experience|work|job|employment|professional)\b",
        "education": r"\b(education|degree|university|college|school)\b"
    }
    section_positions = {}
    for section, pattern in sections.items():
        match = re.search(pattern, text_lower, re.IGNORECASE)
        if match:
            section_positions[section] = match.start()

    structure_feedback = []
    expected_order = ["contact", "summary", "skills", "experience", "education"]
    detected_sections = list(section_positions.keys())
    missing_sections = [s for s in expected_order if s not in detected_sections]
    if missing_sections:
        structure_feedback.append(f"Missing sections: {', '.join(missing_sections)}")
    if detected_sections:
        sorted_sections = sorted(detected_sections, key=lambda x: section_positions[x])
        ideal_order = [s for s in expected_order if s in detected_sections]
        if sorted_sections != ideal_order:
            structure_feedback.append(f"Sections out of order. Detected: {', '.join(sorted_sections)}. Recommended: {', '.join(ideal_order)}")

    detected_skills = []
    for skill in job_skills:
        if re.search(r'\b' + re.escape(skill) + r'\b', text_lower):
            detected_skills.append(skill)
    matched_skills = len(detected_skills)
    total_skills = len(job_skills)
    score = (matched_skills / total_skills) * 100 if total_skills > 0 else 0
    if len(re.findall(r"\b(header|footer|image|table)\b", text_lower)) > 0:
        score -= 20
    score = max(0, min(100, score))
    missing_skills = [skill for skill in job_skills if skill not in detected_skills]

    skill_feedback = []
    critical_skills = job_skills[:5]
    for skill in detected_skills:
        relevance = "Critical" if skill in critical_skills else "Supplementary"
        skill_feedback.append(f"- {skill}: {relevance} skill detected")
    if not detected_skills:
        skill_feedback.append("- No relevant skills detected")

    # Generate a concise summary
    summary_text = f"Your resume scores {score:.1f}% for the {original_job_role} role. "
    if detected_skills:
        summary_text += f"It demonstrates strong skills in {', '.join(detected_skills[:3])}, "
    else:
        summary_text += "It lacks relevant skills for this role, "
    if missing_sections:
        summary_text += f"but is missing key sections like {', '.join(missing_sections)}, which are critical for ATS compatibility."
    else:
        summary_text += "and follows the recommended structure for ATS compatibility."

    # Enhanced interface with polygon background and heading
    color = "#ff4d4d" if score < 40 else "#ffcc00" if score < 70 else "#4caf50"
    circumference = 2 * 3.14159 * 50
    dash_length = (score / 100) * circumference
    combined_output = f"""
    <style>
        /* Main container */
        .main-container {{
            font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, Oxygen, Ubuntu, Cantarell, 'Open Sans', 'Helvetica Neue', sans-serif;
            padding: 30px;
            border-radius: 10px;
            max-width: 800px;
            margin: 0 auto;
        }}

        /* ATS Score Section */
        .ats-score {{
            text-align: center;
            margin-bottom: 40px;
        }}
        .ats-score h3 {{
            font-size: 24px;
            margin-bottom: 15px;
        }}
        @media (prefers-color-scheme: light) {{
            .ats-score h3 {{
                color: #333;
            }}
            .ats-score text {{
                fill: #333;
            }}
        }}
        @media (prefers-color-scheme: dark) {{
            .ats-score h3 {{
                color: #e2e8f0;
            }}
            .ats-score text {{
                fill: #e2e8f0;
            }}
        }}

        /* Report Section */
        .report-box {{
            padding: 25px;
            border-radius: 10px;
            box-shadow: 0 4px 12px rgba(0,0,0,0.1);
        }}
        @media (prefers-color-scheme: light) {{
            .report-box {{
                background: #fff;
            }}
            .report-box h3, .report-box h4, .report-box h5 {{
                color: #2c3e50;
            }}
            .report-box p {{
                color: #555;
            }}
            .report-box strong {{
                color: #34495e;
            }}
        }}
        @media (prefers-color-scheme: dark) {{
            .report-box {{
                background: #2d3748;
            }}
            .report-box h3, .report-box h4, .report-box h5 {{
                color: #e2e8f0;
            }}
            .report-box p {{
                color: #b0b7c3;
            }}
            .report-box strong {{
                color: #d1d5db;
            }}
        }}
    </style>

    <div class="main-container">
        <!-- ATS Score Section -->
        <div class="ats-score">
            <h3>ATS Score: {score:.1f}%</h3>
            <svg width="200" height="200" viewBox="0 0 200 200" style="filter: drop-shadow(0 4px 12px rgba(0,0,0,0.1));">
                <circle cx="100" cy="100" r="50" fill="none" stroke="#e6e6e6" stroke-width="10"/>
                <circle cx="100" cy="100" r="50" fill="none" stroke="{color}" stroke-width="10"
                    stroke-dasharray="{dash_length} {circumference}" stroke-dashoffset="0"
                    transform="rotate(-90 100 100)">
                    <animate attributeName="stroke-dasharray" from="0 {circumference}" to="{dash_length} {circumference}" dur="2s" fill="freeze" />
                </circle>
                <text x="100" y="105" text-anchor="middle" font-size="20">{score:.1f}%</text>
            </svg>
        </div>

        <!-- Detailed Summary Section -->
        <div class="report-box">
            <h3 style="font-size: 22px; margin-bottom: 15px; border-bottom: 2px solid #3498db; padding-bottom: 5px;">Resume Analysis Report</h3>
            <p style="font-size: 16px; margin-bottom: 10px;"><strong>Job Role:</strong> {original_job_role}</p>
            
            <h4 style="font-size: 18px; margin-top: 20px; display: flex; align-items: center;">
                <svg style="margin-right: 8px;" width="20" height="20" viewBox="0 0 24 24" fill="none" stroke="#3498db" stroke-width="2">
                    <path d="M9 12l2 2 4-4m6 2a9 9 0 11-18 0 9 9 0 0118 0z"/>
                </svg>
                Summary
            </h4>
            <p style="font-size: 15px; line-height: 1.6; margin-bottom: 20px;">{summary_text}</p>

            <h5 style="font-size: 16px; margin-top: 20px; display: flex; align-items: center;">
                <svg style="margin-right: 8px;" width="20" height="20" viewBox="0 0 24 24" fill="none" stroke="#3498db" stroke-width="2">
                    <path d="M16 4h2a2 2 0 012 2v14a2 2 0 01-2 2H6a2 2 0 01-2-2V6a2 2 0 012-2h2"/>
                    <rect x="8" y="2" width="8" height="4" rx="1" ry="1"/>
                </svg>
                Resume Structure Analysis
            </h5>
            {"<p style='font-size: 15px; line-height: 1.6;'><strong>Issues Detected:</strong><br>" + "<br>".join([f"  - {f}" for f in structure_feedback]) + "</p><p style='font-size: 15px; line-height: 1.6;'><strong>Recommended Order:</strong> Contact → Summary → Skills → Experience → Education</p>" if structure_feedback else "<p style='font-size: 15px; line-height: 1.6;'><strong>Issues Detected:</strong> None (Structure follows recommended order)</p>"}
            
            {"<h5 style='font-size: 16px; margin-top: 20px; display: flex; align-items: center;'><svg style='margin-right: 8px;' width='20' height='20' viewBox='0 0 24 24' fill='none' stroke='#3498db' stroke-width='2'><path d='M12 8v4m0 4h.01M21 12a9 9 0 11-18 0 9 9 0 0118 0z'/></svg>General Issues</h5><p style='font-size: 15px; line-height: 1.6;'>" + "<br>".join([f"  - {m}" for f in mistakes]) + "</p>" if mistakes else ""}
            
            <h5 style="font-size: 16px; margin-top: 20px; display: flex; align-items: center;">
                <svg style="margin-right: 8px;" width="20" height="20" viewBox="0 0 24 24" fill="none" stroke="#3498db" stroke-width="2">
                    <path d="M9 19c-5 1.5-5-2.5-7-3m14 6v-3.87a3.37 3.37 0 0 0-.94-2.61c3.14-.35 6.44-1.54 6.44-7A5.44 5.44 0 0 0 20 4.77 5.07 5.07 0 0 0 19.91 1S18.73.65 16 2.48a13.38 13.38 0 0 0-7 0C6.27.65 5.09 1 5.09 1A5.07 5.07 0 0 0 5 4.77a5.44 5.44 0 0 0-1.5 3.78c0 5.42 3.3 6.61 6.44 7A3.37 3.37 0 0 0 9 18.13V22"/>
                </svg>
                Skill Relevance Summary
            </h5>
            <p style="font-size: 15px; line-height: 1.6;">{"<br>".join(skill_feedback)}</p>
            
            {"<h5 style='font-size: 16px; margin-top: 20px; display: flex; align-items: center;'><svg style='margin-right: 8px;' width='20' height='20' viewBox='0 0 24 24' fill='none' stroke='#3498db' stroke-width='2'><path d='M5 13l4 4L19 7'/></svg>Suggested Skills to Add (out of " + str(total_skills) + " expected)</h5><p style='font-size: 15px; line-height: 1.6;'>" + "<br>".join([f"  - {s}" for s in missing_skills]) + "</p>" if missing_skills else "<p style='font-size: 15px; line-height: 1.6;'><strong>Suggested Skills to Add:</strong> None</p>"}
        </div>
    </div>
    """

    return combined_output

# Custom CSS to apply the polygon background and updated heading from the start
custom_css = """
@import url('https://fonts.googleapis.com/css2?family=Montserrat:wght@700&display=swap');

/* Polygon background */
body, .gradio-container {
    position: relative;
    overflow-x: hidden;
    min-height: 100vh;
    background: linear-gradient(135deg, #f5f7fa 0%, #d6e4ff 100%);
    will-change: transform;
}
@media (prefers-color-scheme: dark) {
    body, .gradio-container {
        background: linear-gradient(135deg, #1e293b 0%, #334155 100%);
    }
}

/* Polygon elements */
.polygon {
    position: absolute;
    width: 50px;
    height: 50px;
    clip-path: polygon(50% 0%, 100% 50%, 50% 100%, 0% 50%);
    opacity: 0.3;
    z-index: -1;
    will-change: transform;
}
.polygon:nth-child(1) {
    top: 10%;
    left: 5%;
    background: linear-gradient(45deg, #3498db, #8e44ad);
    animation: float 20s infinite linear;
}
.polygon:nth-child(2) {
    top: 30%;
    left: 60%;
    background: linear-gradient(45deg, #8e44ad, #3498db);
    animation: float 25s infinite linear reverse;
}
.polygon:nth-child(3) {
    top: 70%;
    left: 20%;
    background: linear-gradient(45deg, #3498db, #8e44ad);
    animation: float 15s infinite linear;
}
.polygon:nth-child(4) {
    top: 50%;
    left: 80%;
    background: linear-gradient(45deg, #8e44ad, #3498db);
    animation: float 18s infinite linear reverse;
}
@media (prefers-color-scheme: dark) {
    .polygon:nth-child(1), .polygon:nth-child(3) {
        background: linear-gradient(45deg, #60a5fa, #a855f7);
    }
    .polygon:nth-child(2), .polygon:nth-child(4) {
        background: linear-gradient(45deg, #a855f7, #60a5fa);
    }
}

/* Floating animation for polygons */
@keyframes float {
    0% {
        transform: translate(0, 0) rotate(0deg);
    }
    50% {
        transform: translate(100vw, 50vh) rotate(180deg);
    }
    100% {
        transform: translate(0, 0) rotate(360deg);
    }
}

/* Add polygons to the background */
body::before, .gradio-container::before {
    content: '';
    position: absolute;
    top: 0;
    left: 0;
    width: 100%;
    height: 100%;
    z-index: -1;
}
body::after, .gradio-container::after {
    content: '';
    position: absolute;
    top: 0;
    left: 0;
    width: 100%;
    height: 100%;
    z-index: -1;
}

/* Inject polygons dynamically */
.gradio-container::before {
    content: '';
}
.gradio-container::after {
    content: '';
}
.gradio-container {
    position: relative;
}
.gradio-container::before {
    content: '';
    position: absolute;
    top: 0;
    left: 0;
    width: 100%;
    height: 100%;
    z-index: -1;
}
.gradio-container::after {
    content: '';
    position: absolute;
    top: 0;
    left: 0;
    width: 100%;
    height: 100%;
    z-index: -1;
}
.gradio-container > div::before,
.gradio-container > div::after,
.gradio-container > div:nth-child(1)::before,
.gradio-container > div:nth-child(1)::after,
.gradio-container > div:nth-child(2)::before,
.gradio-container > div:nth-child(2)::after,
.gradio-container > div:nth-child(3)::before,
.gradio-container > div:nth-child(3)::after {
    content: '';
    position: absolute;
    width: 50px;
    height: 50px;
    clip-path: polygon(50% 0%, 100% 50%, 50% 100%, 0% 50%);
    opacity: 0.3;
    z-index: -1;
    will-change: transform;
}
.gradio-container > div:nth-child(1)::before {
    top: 10%;
    left: 5%;
    background: linear-gradient(45deg, #3498db, #8e44ad);
    animation: float 20s infinite linear;
}
.gradio-container > div:nth-child(1)::after {
    top: 30%;
    left: 60%;
    background: linear-gradient(45deg, #8e44ad, #3498db);
    animation: float 25s infinite linear reverse;
}
.gradio-container > div:nth-child(2)::before {
    top: 70%;
    left: 20%;
    background: linear-gradient(45deg, #3498db, #8e44ad);
    animation: float 15s infinite linear;
}
.gradio-container > div:nth-child(2)::after {
    top: 50%;
    left: 80%;
    background: linear-gradient(45deg, #8e44ad, #3498db);
    animation: float 18s infinite linear reverse;
}
@media (prefers-color-scheme: dark) {
    .gradio-container > div:nth-child(1)::before,
    .gradio-container > div:nth-child(2)::before {
        background: linear-gradient(45deg, #60a5fa, #a855f7);
    }
    .gradio-container > div:nth-child(1)::after,
    .gradio-container > div:nth-child(2)::after {
        background: linear-gradient(45deg, #a855f7, #60a5fa);
    }
}

/* Custom heading style for Gradio title */
.gradio-container h1 {
    font-family: 'Montserrat', sans-serif !important;
    font-size: 36px !important;
    text-align: center !important;
    margin-bottom: 30px !important;
    position: relative !important;
}
@media (prefers-color-scheme: light) {
    .gradio-container h1 {
        color: #2c3e50 !important;
    }
    .gradio-container h1::after {
        content: '';
        position: absolute;
        width: 80px;
        height: 6px;
        background: linear-gradient(90deg, #3498db, #8e44ad);
        bottom: -15px;
        left: 50%;
        transform: translateX(-50%);
        border-radius: 3px;
        box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
    }
}
@media (prefers-color-scheme: dark) {
    .gradio-container h1 {
        color: #e2e8f0 !important;
    }
    .gradio-container h1::after {
        content: '';
        position: absolute;
        width: 80px;
        height: 6px;
        background: linear-gradient(90deg, #60a5fa, #a855f7);
        bottom: -15px;
        left: 50%;
        transform: translateX(-50%);
        border-radius: 3px;
        box-shadow: 0 2px 4px rgba(0, 0, 0, 0.3);
    }
}

/* Ensure inputs and buttons are visible in both modes */
input, button, .gr-button {
    color: #333 !important;
    background: #fff !important;
    border: 1px solid #ccc !important;
}
@media (prefers-color-scheme: dark) {
    input, button, .gr-button {
        color: #e2e8f0 !important;
        background: #2d3748 !important;
        border: 1px solid #4b5563 !important;
    }
}
"""

interface = gr.Interface(
    fn=gradio_resume_chatbot,
    inputs=[
        gr.File(label="Upload your resume (PDF)"),
        gr.Textbox(label="Enter job role (e.g., Data Scientist, software engineer)")
    ],
    outputs=[
        gr.HTML(label="ATS Score and Detailed Summary")
    ],
    title="Resume Chatbot",
    description="Upload your resume and enter a job role to get an animated ATS score and detailed feedback.",
    css=custom_css
)

interface.launch()

* Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.
